<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/hf_train_my_translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install -q transformers datasets evaluate sacrebleu

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

# Translation

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/1JvfrvZgi6c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

翻译将一种语言的文本序列转换为另一种语言。它是可以将输入转换为输出的多个任务之一，比如翻译或摘要生成等，这是一个强大的框架。翻译系统通常用于不同语言文本之间的翻译，但也可以用于语音或介于文本到语音或语音到文本之间的某种组合，如文本到语音或语音到文本。

本指南将向您展示如何：

1. 官方给的示例数据集没有en-zh 英文到中文的翻译数据子集， 可以用 [Helsinki-NLP/opus-100](https://huggingface.co/datasets/Helsinki-NLP/opus-100)这个数据集，使用en-zh子集，将英文翻译成繁体中文）

「在 [OPUS Books](https://huggingface.co/datasets/opus_books) 数据集的英法子集上对 [T5](https://huggingface.co/t5-small) 进行微调，以将英文文本翻译为法语。」

2. 使用您微调过的模型进行推断。

<提示>
本教程中所示任务受以下模型架构支持：

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bart), [BigBird-Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bigbird_pegasus), [Blenderbot](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot), [BlenderbotSmall](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/blenderbot-small), [编码器解码器](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/encoder-decoder), [FairSeq 机器翻译](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/fsmt), [GPTSAN-japanese](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/gptsan-japanese), [LED](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/led), [LongT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/longt5), [M2M100](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/m2m_100), [Marian](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/marian), [mBART](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mbart), [MT5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mt5), [MVP](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mvp), [NLLB](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb), [NLLB-MOE](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nllb-moe), [Pegasus](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus), [PEGASUS-X](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/pegasus_x), [PLBart](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/plbart), [ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/prophetnet), [SwitchTransformers](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/switch_transformers), [T5](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/t5), [XLM-ProphetNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/xlm-prophetnet)

<!--End of the generated tip-->

</提示>

开始之前，请确保已安装所有必要的库：

```bash
pip install transformers datasets evaluate sacrebleu
```

我们鼓励您登录您的 Hugging Face 账户，这样您就可以上传和共享您的模型给社区。当提示时，请输入您的令牌以登录：

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load OPUS Books dataset

官方给的示例数据集没有en-zh 英文到中文的翻译数据子集， 可以用 [Helsinki-NLP/opus-100](https://huggingface.co/datasets/Helsinki-NLP/opus-100)这个数据集，使用en-zh子集

In [3]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-zh")

ValueError: BuilderConfig 'en-zh' not found. Available: ['ca-de', 'ca-en', 'ca-hu', 'ca-nl', 'de-en', 'de-eo', 'de-es', 'de-fr', 'de-hu', 'de-it', 'de-nl', 'de-pt', 'de-ru', 'el-en', 'el-es', 'el-fr', 'el-hu', 'en-eo', 'en-es', 'en-fi', 'en-fr', 'en-hu', 'en-it', 'en-nl', 'en-no', 'en-pl', 'en-pt', 'en-ru', 'en-sv', 'eo-es', 'eo-fr', 'eo-hu', 'eo-it', 'eo-pt', 'es-fi', 'es-fr', 'es-it', 'es-nl', 'es-no', 'es-pt', 'es-ru', 'fi-fr', 'fi-no', 'fi-pl', 'fr-hu', 'fr-it', 'fr-nl', 'fr-no', 'fr-pl', 'fr-pt', 'fr-ru', 'fr-sv', 'hu-it', 'hu-nl', 'hu-no', 'hu-pl', 'hu-pt', 'hu-ru', 'it-nl', 'it-pt', 'it-ru', 'it-sv']

In [18]:
from datasets import load_dataset

my_books = load_dataset("Helsinki-NLP/opus-100", "en-zh")

使用[train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split)方法将数据集分割为训练集和测试集:

In [19]:
books = my_books["train"].train_test_split(test_size=0.2)

然后看一个例子:

In [20]:
books["train"][0]

{'translation': {'en': 'Take off the extra bits too.', 'zh': '也拿掉其它多余东西'}}

`translation`: 英语到中文翻译。

## Preprocess 预处理

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XAR8jnZZuUs?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

下一步是加载一个T5分词器来处理英语-法语语言对:

In [21]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

你想要创建的预处理函数需要:

1. 用提示符作为输入的前缀，这样T5就知道这是一个翻译任务。一些能够完成多个NLP任务的模型需要对特定任务进行提示。
2. 分别对输入(英语)和目标(中文)进行分词，因为无法使用在英语词汇表上预训练的分词器对中文文本进行分词。
3. 将序列截断为不超过`max_length`参数设置的最大长度。

In [22]:
source_lang = "en"
target_lang = "zh"
prefix = "translate English to Chinese: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

要在整个数据集上应用预处理函数，请使用🤗Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map)方法。你可以通过设置`batched=True`来加速`map`函数，使其一次处理数据集的多个元素:

In [23]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/800000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [24]:
tokenized_books["train"][0]

{'translation': {'en': 'Take off the extra bits too.', 'zh': '也拿掉其它多余东西'},
 'input_ids': [13959, 1566, 12, 2830, 10, 2321, 326, 8, 996, 14120, 396, 5, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [3, 2, 1]}

现在使用 [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq) 创建一个示例批次。在整理过程中，动态地将句子填充到批次中最长的长度会更有效，而不是将整个数据集填充到最大长度。

In [26]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

在训练过程中包含一个度量标准通常有助于评估模型的性能。您可以使用 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) 库快速加载一个评估方法。对于这个任务，加载 [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) 度量标准（查看 🤗 Evaluate [快速入门](https://huggingface.co/docs/evaluate/a_quick_tour) 以了解如何加载和计算度量标准）：

In [27]:
import evaluate

metric = evaluate.load("sacrebleu")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the SacreBLEU score:

In [28]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

你的`compute_metrics`函数现在已经准备好了，当你开始训练的时候就可以回到它。

## Train

如果您还不熟悉如何使用[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)对模型进行微调，请查看此[基础教程](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)！

</Tip>

您现在准备好开始训练您的模型了！使用[AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM)加载T5模型：

如果您对如何使用Hugging Face的Transformers库中的Trainer类来微调模型不太熟悉，建议您参考上述链接提供的基础教程。这个教程将帮助您了解微调过程的基本概念和步骤。
在您熟悉了微调的基础知识之后，就可以开始着手训练您的模型了。首先，您需要加载T5模型，这可以通过使用Transformers库中的AutoModelForSeq2SeqLM类来实现。这个类提供了一个简单的方式来加载预训练的T5模型，它是一个用于序列到序列任务的模型，非常适合用于文本生成任务，如文本摘要、问题回答和翻译等。
通过使用AutoModelForSeq2SeqLM，您可以直接加载T5模型，而无需关心底层的模型架构和权重加载的细节。这样，您就可以专注于您的训练数据和训练过程，而不是模型的加载和配置。
加载模型后，您可以根据您的特定任务和数据集来设置训练参数，然后使用Trainer类来启动训练过程。在训练过程中，Trainer会处理所有的训练细节，包括数据加载、模型更新和评估指标的计算等。
总之，通过结合使用AutoModelForSeq2SeqLM和Trainer，您可以轻松地开始训练自己的T5模型，并将其应用于各种序列到序列的自然语言处理任务。

In [29]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

在这个阶段，只剩下三个步骤需要完成：

1. 在[Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments)中定义你的训练超参数。唯一必需的参数是`output_dir`，它指定了保存模型的位置。通过设置`push_to_hub=True`，你将这个模型推送到Hub上（你需要登录到Hugging Face才能上传你的模型）。在每个时代的结束时，[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)将评估SacreBLEU指标并保存训练检查点。
2. 将训练参数传递给[Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer)，同时还需要提供模型、数据集、分词器、数据整理器以及`compute_metrics`函数。
3. 调用[train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train)来微调你的模型。

通过这三个步骤，你可以有效地进行模型的训练和微调。首先，通过设置Seq2SeqTrainingArguments中的参数，你可以控制模型训练的过程和结果的保存位置。记得设置`push_to_hub`参数，这样训练完成后，你的模型就可以上传到Hugging Face Hub，方便他人使用和分享。
接下来，通过Seq2SeqTrainer，你可以将所有的训练参数、模型、数据集等整合在一起，准备开始训练过程。在这个过程中，你可以利用`compute_metrics`函数来计算和评估模型的性能指标。
最后，通过调用`train()`方法，你的模型将开始微调过程。在这个过程中，模型将根据提供的数据集进行训练，逐步优化参数，以提高在特定任务上的表现。
总的来说，这三个步骤为你提供了一个清晰的训练流程，帮助你高效地完成模型的微调工作。

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.push_to_hub()

如果您希望深入了解如何对模型进行翻译任务的微调，可以参考Hugging Face提供的相应[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb)或者[TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb)。这两个notebook提供了详细的代码示例和步骤说明，帮助您理解微调过程的每个环节。


## Inference

很好，现在你已经对模型进行了微调，你可以使用它进行推理!

想出一些你想翻译成另一种语言的文本。对于T5，您需要根据您正在处理的任务为输入添加前缀。为了从英语翻译到中文，你应该像下面这样在输入前加上前缀:

In [ ]:
text = "translate English to Chinese: Legumes share resources with nitrogen-fixing bacteria."

尝试用于推理的微调模型的最简单方法是在[pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline)中使用它。用你的模型实例化一个用于翻译的`pipeline`，并将你的文本传递给它:

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="my_awesome_opus_books_model")
translator(text)

[{'translation_text': 'Legumes partagent des ressources avec des bactéries azotantes.'}]

如果你愿意，你也可以手动复制`pipeline`的结果:

将文本分词并将`input_ids`作为PyTorch张量返回:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

使用[generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate)方法创建翻译。有关控制生成的不同文本生成策略和参数的更多详细信息，请查看[text generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API。

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

将生成的token id解码为文本:

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)